In [1]:
import requests
import pandas as pd
import concurrent.futures
import time

app_details_url = "https://store.steampowered.com/api/appdetails?appids={}"

url = "http://api.steampowered.com/ISteamApps/GetAppList/v2/"

response = requests.get(url)

def get_app_details(app_id):
    details_response = requests.get(app_details_url.format(app_id))
    if details_response.status_code == 200:
        details = details_response.json()
        if str(app_id) in details:
            data = details[str(app_id)].get("data")
            if data:
                release_date = data.get("release_date", {}).get("date")
                price = data.get("price_overview", {}).get("final_formatted")
                publisher = ", ".join(data.get("publishers", []))
                genres = data.get("genres", [])
                return {"App ID": app_id, "Release Date": release_date, "Price": price, "Publisher": publisher, "Genre": genres}

if response.status_code == 200:
    app_list = response.json()["applist"]["apps"]
    apps = []
    app_ids = [app["appid"] for app in app_list][:1000] # limit the number of apps to 10
    app_names = {app["appid"]: app["name"] for app in app_list}
    start_time = time.time()
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(get_app_details, app_id) for app_id in app_ids]
        for future in concurrent.futures.as_completed(futures):
            app = future.result()
            if app:
                app["App Name"] = app_names[app["App ID"]]
                apps.append(app)
    df = pd.DataFrame(apps)
    df.to_excel("apps.xlsx", index=False)
    print("Data saved successfully to apps.xlsx")
else:
    print("Request failed with status code: {}".format(response.status_code))
